<a href="https://colab.research.google.com/github/zainazeem41472/Rice-Leaf-Disease-Classification/blob/main/experimentcnn4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.19.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Dropout,
    BatchNormalization, Input, GlobalAveragePooling2D,
    RandomFlip, RandomRotation, RandomZoom, RandomContrast, Rescaling
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

DATASET_DIR = "/content/drive/MyDrive/RiceLeafsDisease"
TRAIN_DIR = DATASET_DIR + "/train"
VAL_DIR   = DATASET_DIR + "/validation"

img_size = 224
batch_size = 32
num_classes = 6


In [5]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


In [6]:
data_augmentation = tf.keras.Sequential([
    Rescaling(1./255),
    RandomFlip("horizontal"),
    RandomRotation(0.15),
    RandomZoom(0.2),
    RandomContrast(0.2),
])


In [7]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    image_size=(img_size, img_size),
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR,
    image_size=(img_size, img_size),
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=False
)

print("Classes:", train_ds.class_names)


Found 2100 files belonging to 6 classes.
Found 528 files belonging to 6 classes.
Classes: ['bacterial_leaf_blight', 'brown_spot', 'healthy', 'leaf_blast', 'leaf_scald', 'narrow_brown_spot']


In [8]:
activation = tf.keras.activations.hard_sigmoid

inputs = Input(shape=(img_size, img_size, 3))
x = data_augmentation(inputs)

# -------- Block 1 --------
x = Conv2D(32, 3, padding='same', activation=activation)(x)
x = BatchNormalization()(x)
x = Conv2D(32, 3, padding='same', activation=activation)(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)

# -------- Block 2 --------
x = Conv2D(64, 3, padding='same', activation=activation)(x)
x = BatchNormalization()(x)
x = Conv2D(64, 3, padding='same', activation=activation)(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)

# -------- Block 3 --------
x = Conv2D(128, 3, padding='same', activation=activation)(x)
x = BatchNormalization()(x)
x = Conv2D(128, 3, padding='same', activation=activation)(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)

# -------- Block 4 --------
x = Conv2D(256, 3, padding='same', activation=activation)(x)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)

# -------- Classifier --------
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation=activation)(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs, output)


In [13]:
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=7,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=3,
    min_lr=1e-6
)


In [17]:
model.compile(
    optimizer=Adam(learning_rate=0.0003),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [18]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[early_stop, reduce_lr]
)


Epoch 1/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 555s 8s/step - accuracy: 0.5314 - loss: 1.4797 - val_accuracy: 0.1667 - val_loss: 1.9220 - learning_rate: 3.0000e-04
Epoch 2/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 27s 403ms/step - accuracy: 0.5976 - loss: 1.1929 - val_accuracy: 0.1667 - val_loss: 1.8522 - learning_rate: 3.0000e-04
Epoch 3/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 27s 407ms/step - accuracy: 0.6243 - loss: 1.0990 - val_accuracy: 0.1667 - val_loss: 1.8482 - learning_rate: 3.0000e-04
Epoch 4/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 31s 475ms/step - accuracy: 0.6628 - loss: 0.9912 - val_accuracy: 0.1667 - val_loss: 1.9899 - learning_rate: 3.0000e-04
Epoch 5/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 27s 403ms/step - accuracy: 0.6831 - loss: 0.9462 - val_accuracy: 0.1667 - val_loss: 2.5286 - learning_rate: 3.0000e-04
Epoch 6/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 27s 400ms/step - accuracy: 0.7036 - loss: 0.8938 - val_accuracy: 0.1667 - val_loss: 3.2899 - learning_rate: 3.0000e-04
Epoch 7/50
66/66 ━━━━━━━━━━━━━━━━━━━━ 27s 405ms/step - accur

In [19]:
print(f"Training Accuracy: {history.history['accuracy'][-1]*100:.2f}%")
print(f"Validation Accuracy: {history.history['val_accuracy'][-1]*100:.2f}%")


Training Accuracy: 76.90%
Validation Accuracy: 16.67%
